In [94]:
"""Example of Estimator for CNN-based text classification with DBpedia data."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf

result = []

## Set hyperparameter

In [101]:
FLAGS = None

MAX_DOCUMENT_LENGTH = 100
EMBEDDING_SIZE = 20
N_FILTERS = 256
WINDOW_SIZE = 20
FILTER_SHAPE1 = [WINDOW_SIZE, EMBEDDING_SIZE]
FILTER_SHAPE2 = [WINDOW_SIZE, N_FILTERS]
POOLING_WINDOW = 4
POOLING_STRIDE = 2
n_words = 0
MAX_LABEL = 4
WORDS_FEATURE = 'words'  # Name of the input words feature.

In [102]:
def cnn_model(features, labels, mode):
    word_vectors = tf.contrib.layers.embed_sequence(
        features[WORDS_FEATURE],
        vocab_size=n_words,
        embed_dim=EMBEDDING_SIZE)
    word_vectors = tf.expand_dims(word_vectors, 3)
    
    with tf.variable_scope('CNN_Layer1'):
        # Apply Convolution filtering on input sequence.
        conv1 = tf.layers.conv2d(
            word_vectors,
            filters=N_FILTERS,
            kernel_size=FILTER_SHAPE1,
            padding='VALID',
            # Add a ReLU for non linearity.
            activation=tf.nn.relu)
        # Max pooling across output of Convolution+Relu.
        pool1 = tf.layers.max_pooling2d(
            conv1,
            pool_size=POOLING_WINDOW,
            strides=POOLING_STRIDE,
            padding='SAME')
        # Transpose matrix so that n_filters from convolution becomes width.
        pool1 = tf.transpose(pool1, [0, 1, 3, 2])
        
    with tf.variable_scope('CNN_Layer2'):
        # Second level of convolution filtering.
        conv2 = tf.layers.conv2d(
            pool1,
            filters=N_FILTERS,
            kernel_size=FILTER_SHAPE2,
            padding='VALID')
        
        # Max across each filter to get useful features for classification.
        pool2 = tf.squeeze(tf.reduce_max(conv2, 1), squeeze_dims=[1])
        
    # Apply regular WX + B and classification.
    logits = tf.layers.dense(pool2, MAX_LABEL, activation=None)
    
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'class': predicted_classes,
                'prob': tf.nn.softmax(logits)
            })

    onehot_labels = tf.one_hot(labels, MAX_LABEL, 1, 0)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels,
        logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

    eval_metric_ops = {'accuracy': tf.metrics.accuracy(
            labels=labels,
            predictions=predicted_classes)}
    
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        eval_metric_ops=eval_metric_ops)

## Data processing

In [103]:
train_csv = pd.read_csv('pubmed_data/train.csv', sep='::', header=None)
x_train = train_csv[1]
y_train = train_csv[0]

/Users/yoonjae/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [104]:
test_csv = pd.read_csv('pubmed_data/test.csv', sep='::', header=None)
x_test = test_csv[1]
y_test = test_csv[0]

/Users/yoonjae/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


## Main flow

In [105]:
global n_words

In [106]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--test_with_fake_data',
    default=False,
    help='Test the example code with fake data.',
    action='store_true')
FLAGS, unparsed = parser.parse_known_args()

In [107]:
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
x_train = np.array(list(vocab_processor.fit_transform(x_train)))
x_test = np.array(list(vocab_processor.transform(x_test)))
n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

Total words: 14203


In [108]:
classifier = tf.estimator.Estimator(model_fn=cnn_model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/xw/tcgt4j055nv_cgcjrgjx4tz00000gn/T/tmphe226eio', '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_tf_random_seed': 1, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_session_config': None}


## Train

In [109]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={WORDS_FEATURE: x_train},
    y=y_train,
    batch_size=len(x_train),
    num_epochs=None,
    shuffle=True)
classifier.train(input_fn=train_input_fn, steps=100)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/xw/tcgt4j055nv_cgcjrgjx4tz00000gn/T/tmphe226eio/model.ckpt.
INFO:tensorflow:step = 1, loss = 1.11726
INFO:tensorflow:Saving checkpoints for 36 into /var/folders/xw/tcgt4j055nv_cgcjrgjx4tz00000gn/T/tmphe226eio/model.ckpt.
INFO:tensorflow:Saving checkpoints for 74 into /var/folders/xw/tcgt4j055nv_cgcjrgjx4tz00000gn/T/tmphe226eio/model.ckpt.
INFO:tensorflow:Saving checkpoints for 100 into /var/folders/xw/tcgt4j055nv_cgcjrgjx4tz00000gn/T/tmphe226eio/model.ckpt.
INFO:tensorflow:Loss for final step: 177873.0.


## Predict

In [110]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={WORDS_FEATURE: x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
predictions = classifier.predict(input_fn=test_input_fn)
y_predicted = np.array(list(p['class'] for p in predictions))
y_predicted = y_predicted.reshape(np.array(y_test).shape)

INFO:tensorflow:Restoring parameters from /var/folders/xw/tcgt4j055nv_cgcjrgjx4tz00000gn/T/tmphe226eio/model.ckpt-100


In [111]:
# Score with sklearn.
score = metrics.accuracy_score(y_test, y_predicted)
print('Accuracy (sklearn): {0:f}'.format(score))

# Score with tensorflow.
scores = classifier.evaluate(input_fn=test_input_fn)
print('Accuracy (tensorflow): {0:f}'.format(scores['accuracy']))

Accuracy (sklearn): 0.437158
INFO:tensorflow:Starting evaluation at 2017-07-05-04:24:59
INFO:tensorflow:Restoring parameters from /var/folders/xw/tcgt4j055nv_cgcjrgjx4tz00000gn/T/tmphe226eio/model.ckpt-100
INFO:tensorflow:Finished evaluation at 2017-07-05-04:25:00
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.437158, global_step = 100, loss = 380046.0
Accuracy (tensorflow): 0.437158


In [112]:
result.append(
    {'MAX_DOCUMENT_LENGTH':MAX_DOCUMENT_LENGTH,
     'EMBEDDING_SIZE':EMBEDDING_SIZE,
     'N_FILTERS':N_FILTERS,
     'WINDOW_SIZE':WINDOW_SIZE,
     'POOLING_WINDOW':POOLING_WINDOW,
     'POOLING_STRIDE':POOLING_STRIDE,
     'n_words':n_words,
     'accuracy':scores['accuracy']})

In [113]:
df = pd.DataFrame(result)
df

,EMBEDDING_SIZE,MAX_DOCUMENT_LENGTH,N_FILTERS,POOLING_STRIDE,POOLING_WINDOW,WINDOW_SIZE,accuracy,n_words
0,20,100,128,2,4,20,0.431694,14203
1,20,100,256,2,4,20,0.437158,14203
